In [1]:
import numpy as np
import time
import Mpnet_training
import generate_steering
from typing import Optional, Tuple
import utils_function
import torch
from pathlib import Path
import plots
import mpnet_model
import MP_MpNet_LS_estimation


Using device: cuda


In [2]:
start_time = time.time()

######model hyperparams######
optimizer='Adam'
lr = 0.01
lr_constrained=1e-5
momentum = 0.9
epochs = 1
batch_size = 300
k = 8
batch_subsampling = 10

#Channels generation parameters
nb_channels_test = 1000
nb_channels_train=100
nb_channels_val = 500
nb_channels_per_batch=100
nb_BS_antenna=64


#imperfect config parameters
snr_in_dB = 10 
train_type = 'Online'

#scene generation params
BS_position=[-302, 42, 23.0]
f0=28e9 #HZ

sigma_ant= 0.1
noise_var= 1e-3

# RF chains 
L = 10
T = 1


In [3]:
#get real and nominal antenna pos
path_init=Path.cwd()
file_name = r'Data/datasionna/antenna_position.npz'  
antenna_pos = np.load(path_init/file_name)
nominal_ant_positions=antenna_pos['nominal_position']
real_ant_positions=antenna_pos['real_position']


In [4]:
######Dictionnary parameters######
#random Azimuth angles uniformly distributed between 0 and 2*pi
DoA= np.load(path_init/'Data/DoA.npz')['DoA']
g_vec= np.ones(nb_BS_antenna)
lambda_ =  0.010706874


In [5]:
model = Mpnet_training.UnfoldingModel_Sim(
                 BS_position,
                 nominal_ant_positions,
                 real_ant_positions,
                 sigma_ant,
                 DoA,
                 g_vec,
                 lambda_,
                 snr_in_dB,                  
                 lr,
                 lr_constrained,
                 momentum,                 
                 optimizer,
                 epochs, batch_size,
                 k, 
                 batch_subsampling,
                 train_type,
                 f0,
                 nb_channels_test,
                 nb_channels_train,
                 nb_channels_val,
                 nb_channels_per_batch,
                 nb_BS_antenna,
                 L,
                 T,
                 noise_var
)

In [ ]:
model.train_online_test_inference()  

In [8]:
m = {}
m[f'{T}'] = model.cost_func_test_c
# Save data
np.savez(path_init / 'NMSE_L_T' / f'T_{T}.npz', **m)


In [7]:
m = {}
m['real'] = model.cost_func_mp_real
# Save data
np.savez(path_init / 'NMSE_L_T' / f'real.npz', **m)


In [9]:
m = {}
m['ls'] = model.cost_func_ls
# Save data
np.savez(path_init / 'NMSE_L_T' / f'ls.npz', **m)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
vec_base = np.arange(0,model.batch_size/model.batch_subsampling,1)
#plt.plot(vec_base,model.cost_func_test,'p-',linewidth=0.8,label='mpNet')  
plt.plot(vec_base,model.cost_func_test_c[:batch_size],'-',linewidth=1.3,label='mpNet Constrained ')
plt.plot(vec_base,model.cost_func_mp_nominal[:batch_size],'-',linewidth=1.3,label='MP (Nominal dictionnary) ')
plt.plot(vec_base,model.cost_func_mp_real[:batch_size],'-',linewidth=1.3,label='MP (Real dictionnary) ')  
#plt.plot(vec_base,self.cost_func_ls,'o--',linewidth=0.8,label='LS')  
plt.grid()
plt.legend(loc = 'best')
plt.xlabel(f'Number of seen channels (*{model.batch_subsampling*model.nb_channels_per_batch})')
plt.ylabel('NMSE')
plt.title(f'NMSE evolution: L={L}, T={T},  $\\sigma^2$ = 1e-3, SNR_Avg=10 dB')
plt.xlim(left=0)
plt.xlim(right=29)

In [ ]:
#--------------MPnET_MP_LS_estimation--------------
L=24
model_="Data/mpnet_L_24_T_1_noise_1e-3.pth"
data_file='Data/1e-3/data_var_snr'
data_pred_mpnet=f'Data/1e-3/Estimation/L_{L}_T_{1}/mpnet'
data_pred_MP_nominal='Data/1e-3/Estimation/nominal'
data_pred_MP_real='Data/1e-3/Estimation/real'
data_pred_LS='Data/1e-3/Estimation/ls'
batch_size_est=300

#construct nominal dictionnary 
dict_nominal=generate_steering.steering_vect_c(torch.tensor(nominal_ant_positions).type(torch.FloatTensor),
                                                       torch.tensor(DoA).type(torch.FloatTensor),
                                                       torch.tensor(g_vec),
                                                       lambda_)
#construct real dictionnary 
dict_real   =generate_steering.steering_vect_c(torch.tensor(real_ant_positions).type(torch.FloatTensor),
                                                       torch.tensor(DoA).type(torch.FloatTensor),
                                                       torch.tensor(g_vec),
                                                       lambda_)



MP_MpNet_LS_estimation.save_estimation(model_,data_file,data_pred_mpnet,data_pred_MP_nominal,data_pred_MP_real,data_pred_LS,batch_size_est,k,T,L,dict_nominal,dict_real,
                                       nominal_ant_positions,DoA,g_vec,lambda_,nb_BS_antenna)

